# **Support Vector Machine model for Diabetes Prediction**

In [ ]:
!git clone https://github.com/blaise-fonguh/Diabetes-Prediction-Using-MachineLearning.git

Cloning into 'Diabetes-Prediction-Using-MachineLearning'...
remote: Enumerating objects: 46, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 46 (delta 14), reused 31 (delta 6), pack-reused 0
Receiving objects: 100% (46/46), 34.63 KiB | 11.54 MiB/s, done.
Resolving deltas: 100% (14/14), done.


Importing the Dependencies

In [ ]:
import numpy as np
import math
import joblib
from sklearn import datasets
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.svm import SVC
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from flask import Flask, render_template, request
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

Data Collection and Analysis
PIMA Diabetes Dataset

In [ ]:
# loading the diabetes dataset to a pandas Dataframe
diabetes_dataset = pd.read_csv('/content/diabetes data.csv')


In [ ]:
pd.read_csv

<function pandas.io.parsers.readers.read_csv(filepath_or_buffer: 'FilePath | ReadCsvBuffer[bytes] | ReadCsvBuffer[str]', *, sep: 'str | None | lib.NoDefault' = <no_default>, delimiter: 'str | None | lib.NoDefault' = None, header: "int | Sequence[int] | None | Literal['infer']" = 'infer', names: 'Sequence[Hashable] | None | lib.NoDefault' = <no_default>, index_col: 'IndexLabel | Literal[False] | None' = None, usecols=None, squeeze: 'bool | None' = None, prefix: 'str | lib.NoDefault' = <no_default>, mangle_dupe_cols: 'bool' = True, dtype: 'DtypeArg | None' = None, engine: 'CSVEngine | None' = None, converters=None, true_values=None, false_values=None, skipinitialspace: 'bool' = False, skiprows=None, skipfooter: 'int' = 0, nrows: 'int | None' = None, na_values=None, keep_default_na: 'bool' = True, na_filter: 'bool' = True, verbose: 'bool' = False, skip_blank_lines: 'bool' = True, parse_dates=None, infer_datetime_format: 'bool' = False, keep_date_col: 'bool' = False, date_parser=None, dayf

Exploring the dataset

In [ ]:
# printing the first 5 rows of the dataset
diabetes_dataset.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [ ]:
# Get information about data types and missing values
diabetes_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [ ]:
# number of rows and Columns in this dataset
diabetes_dataset.shape

(768, 9)

In [ ]:
# getting the statisitcal measures of the data
diabetes_dataset.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


Handling missing values

In [ ]:
# Handle missing values (if necessary)
diabetes_dataset.fillna(diabetes_dataset.mean(), inplace=True)    # Filling missing values with the mean

Handling Outliers

In [ ]:
def find_outliers(diabetes_dataset):
    # Flatten the matrix into a 1D array
    flattened_matrix = np.array(diabetes_dataset).flatten()

    # Calculate the mean and standard deviation of the elements in the matrix
    mean = np.mean(flattened_matrix)
    std_dev = np.std(flattened_matrix)

    # Define a threshold for outliers (e.g., 2 standard deviations away from the mean)
    threshold = 2 * std_dev

    # Find outliers in the matrix using the Z-score method
    outliers = [element for element in flattened_matrix if abs(element - mean) > threshold]

    return outliers

# Find outliers in the matrix
outliers = find_outliers(diabetes_dataset)
print("Outliers in our dataset:", outliers)

Outliers in our dataset: [183.0, 168.0, 197.0, 543.0, 168.0, 189.0, 846.0, 166.0, 175.0, 230.0, 235.0, 196.0, 158.0, 245.0, 192.0, 207.0, 180.0, 171.0, 240.0, 159.0, 180.0, 176.0, 300.0, 342.0, 187.0, 304.0, 270.0, 176.0, 228.0, 220.0, 163.0, 171.0, 155.0, 495.0, 160.0, 175.0, 162.0, 173.0, 168.0, 170.0, 225.0, 325.0, 154.0, 284.0, 204.0, 156.0, 155.0, 485.0, 188.0, 163.0, 285.0, 156.0, 179.0, 194.0, 181.0, 495.0, 160.0, 159.0, 158.0, 210.0, 318.0, 190.0, 196.0, 280.0, 162.0, 184.0, 179.0, 175.0, 271.0, 177.0, 478.0, 158.0, 190.0, 162.0, 197.0, 744.0, 370.0, 171.0, 181.0, 192.0, 179.0, 164.0, 176.0, 194.0, 184.0, 165.0, 680.0, 402.0, 258.0, 193.0, 375.0, 155.0, 191.0, 278.0, 155.0, 161.0, 155.0, 545.0, 220.0, 182.0, 194.0, 161.0, 360.0, 215.0, 184.0, 167.0, 161.0, 180.0, 205.0, 155.0, 182.0, 194.0, 231.0, 157.0, 168.0, 156.0, 179.0, 180.0, 165.0, 255.0, 171.0, 178.0, 165.0, 167.0, 196.0, 249.0, 189.0, 325.0, 158.0, 293.0, 173.0, 465.0, 158.0, 325.0, 156.0, 182.0, 285.0, 196.0, 166.0, 4

In [ ]:
diabetes_dataset['Outcome'].value_counts()

0    500
1    268
Name: Outcome, dtype: int64

0 --> Non-Diabetic
1 --> Diabetic


In [ ]:
diabetes_dataset.groupby('Outcome').mean()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
Outcome,,,,,,,,
0,3.298000,109.980000,68.184000,19.664000,68.792000,30.304200,0.429734,31.190000
1,4.865672,141.257463,70.824627,22.164179,100.335821,35.142537,0.550500,37.067164


In [ ]:
# seperating the data and labels
X = diabetes_dataset.drop(columns = 'Outcome', axis=1)
Y = diabetes_dataset['Outcome']


In [ ]:
print(X)

     Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0              6      148             72             35        0  33.6   
1              1       85             66             29        0  26.6   
2              8      183             64              0        0  23.3   
3              1       89             66             23       94  28.1   
4              0      137             40             35      168  43.1   
..           ...      ...            ...            ...      ...   ...   
763           10      101             76             48      180  32.9   
764            2      122             70             27        0  36.8   
765            5      121             72             23      112  26.2   
766            1      126             60              0        0  30.1   
767            1       93             70             31        0  30.4   

     DiabetesPedigreeFunction  Age  
0                       0.627   50  
1                       0.351   31  


In [ ]:
print(Y)

0      1
1      0
2      1
3      0
4      1
      ..
763    0
764    0
765    0
766    1
767    0
Name: Outcome, Length: 768, dtype: int64


Data Standardization

In [ ]:
scaler = StandardScaler()

In [ ]:
scaler.fit(X)
StandardScaler(copy=True, with_mean=True, with_std=True)

StandardScaler()

In [ ]:
standardized_data = scaler.transform(X)

In [ ]:
print(standardized_data)

[[ 0.63994726  0.84832379  0.14964075 ...  0.20401277  0.46849198
   1.4259954 ]
 [-0.84488505 -1.12339636 -0.16054575 ... -0.68442195 -0.36506078
  -0.19067191]
 [ 1.23388019  1.94372388 -0.26394125 ... -1.10325546  0.60439732
  -0.10558415]
 ...
 [ 0.3429808   0.00330087  0.14964075 ... -0.73518964 -0.68519336
  -0.27575966]
 [-0.84488505  0.1597866  -0.47073225 ... -0.24020459 -0.37110101
   1.17073215]
 [-0.84488505 -0.8730192   0.04624525 ... -0.20212881 -0.47378505
  -0.87137393]]


In [ ]:
X = standardized_data
Y = diabetes_dataset['Outcome']

In [ ]:
print(X)
print(Y)


[[ 0.63994726  0.84832379  0.14964075 ...  0.20401277  0.46849198
   1.4259954 ]
 [-0.84488505 -1.12339636 -0.16054575 ... -0.68442195 -0.36506078
  -0.19067191]
 [ 1.23388019  1.94372388 -0.26394125 ... -1.10325546  0.60439732
  -0.10558415]
 ...
 [ 0.3429808   0.00330087  0.14964075 ... -0.73518964 -0.68519336
  -0.27575966]
 [-0.84488505  0.1597866  -0.47073225 ... -0.24020459 -0.37110101
   1.17073215]
 [-0.84488505 -0.8730192   0.04624525 ... -0.20212881 -0.47378505
  -0.87137393]]
0      1
1      0
2      1
3      0
4      1
      ..
763    0
764    0
765    0
766    1
767    0
Name: Outcome, Length: 768, dtype: int64


Train Test Split

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, stratify=Y, random_state=2)

In [ ]:
print(X.shape, X_train.shape, X_test.shape)

(768, 8) (614, 8) (154, 8)


Training the model and hyperparameter tuning

In [ ]:
classifier = svm.SVC(kernel='linear')

In [ ]:
# Perform hyperparameter tuning using GridSearchCV
param_grid = {'C': [0.1, 1, 10, 100], 'kernel': ['linear', 'rbf']}
grid_search = GridSearchCV(classifier, param_grid, cv=5)
grid_search.fit(X_train, Y_train)
best_params = grid_search.best_params_

In [ ]:
#training the support vector Machine Classifier
svm_model = SVC(**best_params)
classifier.fit(X_train, Y_train)

SVC(kernel='linear')

Model Evaluation

Accuracy Score

In [ ]:
# accuracy score on the training data
X_train_prediction = classifier.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [ ]:
print('Accuracy score of the training data : ', training_data_accuracy )

Accuracy score of the training data :  0.7866449511400652


In [ ]:
# accuracy score on the test data
X_test_prediction = classifier.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [ ]:
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.7727272727272727


Precision score

In [ ]:
# precision score on test data
X_test_prediction = classifier.predict(X_test)
test_data_precision = precision_score(X_test_prediction, Y_test)
print('Precision score of the test data : ', test_data_precision)

Precision score of the test data :  0.5185185185185185


Recall score

In [ ]:
# recall score on test data
test_data_recall = recall_score(X_test_prediction, Y_test)
print('Recall score of the test data : ', test_data_recall)

Recall score of the test data :  0.7567567567567568


F1 score

In [ ]:
# f1 score on test data
test_data_f1 = f1_score(X_test_prediction, Y_test)
print('F1 score of the test data : ', test_data_f1)

F1 score of the test data :  0.6153846153846154


Confusion Matrix

In [ ]:
# confusion matrix on test data
test_data_confusion_mat = confusion_matrix(X_test_prediction, Y_test)
print('Confusion Matrix of the test data : ', test_data_confusion_mat)

Confusion Matrix of the test data :  [[91 26]
 [ 9 28]]


Making a Predictive System

In [ ]:
input_data = (3,158,76,36,245,31.6,0.851,28)

# changing the input_data to numpy array
input_data_as_numpy_array = np.asarray(input_data)

# reshape the array as we are predicting for one instance
input_data_reshaped = input_data_as_numpy_array.reshape(1, -1)

#standardize the input data
std_data = scaler.transform(input_data_reshaped)
print(std_data)

prediction = classifier.predict(std_data)
print(prediction)

if (prediction[0] == 0):
   print('The person is not diabetic')
else:
   print('The person is diabetic')

[[-0.25095213  1.16129525  0.35643175  0.96999799  1.43441893 -0.04982572
   1.14499856 -0.44593516]]
[1]
The person is diabetic


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [ ]:
joblib.dump(classifier, 'diabetes_prediction_model.joblib')

['diabetes_prediction_model.joblib']

Building a user interface for the system

In [ ]:
import pickle
import flask
import numpy as np
from flask import Flask, render_template, request
import joblib
import sklearn.svm
from sklearn.preprocessing import StandardScaler

app = Flask(__name__)

# Load the machine learning model
model = joblib.load('diabetes_prediction_model.joblib')

@app.route("/")
def index():
    form = """
    <form action="/predict" method="post">
        <input type="number" name="Pregnancies" placeholder="Number of pregnancies">
        <input type="number" name="glucose" placeholder="Glucose level">
        <input type="number" name="blood_pressure" placeholder="Blood pressure">
        <input type="number" name="skin_thickness" placeholder="Skin thickness">
        <input type="number" name="insulin" placeholder="Insulin level">
        <input type="number" name="bmi" placeholder="BMI">
        <input type="number" name="dpf" placeholder="DiabetesPedigreeFunction">
        <input type="number" name="age" placeholder="Age">
        <input type="submit" value="Predict">
    </form>
    """
    return form

@app.route("/predict", methods=["POST"])
def predict():
    # Get the user input data
    pregnancies = flask.request.form.get("pregnancies")
    glucose = flask.request.form.get("glucose")
    blood_pressure = flask.request.form.get("blood_pressure")
    skin_thickness = flask.request.form.get("skin_thickness")
    insulin = flask.request.form.get("insulin")
    bmi = flask.request.form.get("bmi")
    dpf = flask.request.form.get("dpf")
    age = flask.request.form.get("age")

    # Standardize the input data
    input_data = np.array([pregnancies, glucose, blood_pressure, skin_thickness, insulin, bmi, dpf, age])
    scaler = StandardScaler()
    scaler.fit(input_data)
    scaled_data = scaler.transform(input_data)

    # Reshape the input data
    scaled_data = scaled_data.reshape(1, -1)

    # Predict whether the user is diabetic
    prediction = model.predict_proba(scaled_data)[0][1]

    # Return the prediction
    return "The probability of you being diabetic is {}%".format(round(prediction * 100, 2))

if __name__ == "__main__":
  app.run(debug=True)